<a href="https://colab.research.google.com/github/Jih00nJung/assignment_list/blob/main/GAN_FMNIST_v3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

v3.2의 배치사이즈를 16 -> 128으로 변경

1. config.py (설정 및 환경 준비)

In [7]:
%%writefile config.py
import os
import argparse
from torchvision import transforms
from google.colab import drive

def get_config():
    """학습에 필요한 모든 하이퍼파라미터를 정의합니다."""
    parser = argparse.ArgumentParser()

    # 데이터 및 경로 설정
    parser.add_argument('--project_name', type=str, default='GAN_FMNIST_v3-2b64')
    parser.add_argument('--save_root', type=str, default='/content/drive/MyDrive/Colab Notebooks/GAN_assignment')
    parser.add_argument('--img_size', type=int, default=64, help='이미지 크기 (FMNIST 기본 28 -> 64 리사이즈)')
    parser.add_argument('--batch_size', type=int, default=64)

    # 모델 하이퍼파라미터
    parser.add_argument('--style_dim', type=int, default=64, help='스타일 코드 차원')
    parser.add_argument('--latent_dim', type=int, default=16, help='랜덤 노이즈 차원')
    parser.add_argument('--num_domains', type=int, default=10, help='Fashion MNIST 클래스 개수')
    parser.add_argument('--hidden_dim', type=int, default=256, help='Mapping Network 히든 차원')

    # 학습 설정
    parser.add_argument('--total_iters', type=int, default=100000)
    parser.add_argument('--resume_iter', type=int, default=0)
    parser.add_argument('--lr', type=float, default=1e-4)
    parser.add_argument('--w_hpf', type=float, default=1, help='High-pass filtering 가중치')
    parser.add_argument('--d_train_repeats', type=int, default=5, help='Discriminator 학습 반복 횟수') # 추가된 부분

    # Loss 가중치
    parser.add_argument('--lambda_sty', type=float, default=5.0)
    parser.add_argument('--lambda_ds', type=float, default=5.0)
    parser.add_argument('--lambda_cyc', type=float, default=0)
    parser.add_argument('--lambda_r1', type=float, default=1.0, help='R1 regularization loss 가중치') # R1 loss 가중치 추가

    # 로깅 주기
    parser.add_argument('--sample_freq', type=int, default=1000)
    parser.add_argument('--save_freq', type=int, default=5000)

    # FID/LPIPS 평가용 설정
    parser.add_argument('--num_fid_samples', type=int, default=1000, help='FID 계산에 사용할 생성 이미지 수')

    args, _ = parser.parse_known_args()
    return args

def prepare_environment(args):
    """Google Drive 마운트 및 체크포인트/샘플 디렉토리를 생성합니다."""
    print("--- 환경 설정 중 ---")
    save_path = os.path.join(args.save_root, args.project_name)

    # Google Drive 마운트
    if not os.path.exists('/content/drive'):
        print("Google Drive를 마운트합니다...")
        drive.mount('/content/drive')
        print("Google Drive mounted.")

    os.makedirs(os.path.join(save_path, 'checkpoints'), exist_ok=True)
    os.makedirs(os.path.join(save_path, 'samples'), exist_ok=True)
    print(f"저장 경로: {save_path}")
    return save_path

def get_data_transform(img_size):
    """Fashion MNIST 데이터 전처리를 정의합니다."""
    return transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,)) # [0,1] -> [-1,1]
    ])

def get_domain_labels():
    """Fashion MNIST의 10개 도메인 레이블을 반환합니다."""
    return ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
            'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


Overwriting config.py


2. model.py (네트워크 아키텍처)

In [8]:
%%writefile model.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdaIN(nn.Module):
    """Adaptive Instance Normalization"""
    def __init__(self, style_dim, num_features):
        super().__init__()
        # 1. 정규화 도구 (학습 파라미터 없음, 단순 통계 정규화)
        self.norm = nn.InstanceNorm2d(num_features, affine=False)
        # 2. 스타일 코드 s를 변환하여 감마(스케일)와 베타(시프트)를 만드는 선형 층
        self.fc = nn.Linear(style_dim, num_features * 2)

    def forward(self, x, s):
        # s를 통해 파라미터 생성 (h)
        h = self.fc(s)
        h = h.view(h.size(0), h.size(1), 1, 1)
        # 생성된 파라미터를 감마와 베타로 나눔
        gamma, beta = torch.chunk(h, chunks=2, dim=1)
        # 정규화된 x에 감마를 곱하고 베타를 더함 -> 스타일 주입
        return (1 + gamma) * self.norm(x) + beta

class ResBlock(nn.Module):
    """기본 ResBlock (다운샘플링 블록에서 사용)"""
    def __init__(self, dim_in, dim_out, actv=nn.LeakyReLU(0.2)):
        super().__init__()
        self.main = nn.Sequential(
            actv,
            nn.Conv2d(dim_in, dim_out, 3, 1, 1),
            nn.InstanceNorm2d(dim_out, affine=True),
            actv,
            nn.Conv2d(dim_out, dim_out, 3, 1, 1),
            nn.InstanceNorm2d(dim_out, affine=True)
        )
        self.shortcut = nn.Sequential()
        if dim_in != dim_out:
            self.shortcut = nn.Conv2d(dim_in, dim_out, 1, 1, 0)

    def forward(self, x):
        return self.main(x) + self.shortcut(x)

class AdaINResBlock(nn.Module):
    """Generator용 AdaIN ResBlock (Bottleneck 및 Up-sampling 블록에서 사용)"""
    def __init__(self, dim_in, dim_out, style_dim, actv=nn.LeakyReLU(0.2)):
        super().__init__()
        self.actv = actv
        self.conv1 = nn.Conv2d(dim_in, dim_out, 3, 1, 1)
        self.conv2 = nn.Conv2d(dim_out, dim_out, 3, 1, 1)
        self.norm1 = AdaIN(style_dim, dim_in)
        self.norm2 = AdaIN(style_dim, dim_out)

        self.shortcut = nn.Sequential()
        if dim_in != dim_out:
            self.shortcut = nn.Conv2d(dim_in, dim_out, 1, 1, 0)

    def forward(self, x, s):
        out = self.norm1(x, s)
        out = self.actv(out)
        out = self.conv1(out)
        out = self.norm2(out, s)
        out = self.actv(out)
        out = self.conv2(out)
        return out + self.shortcut(x)

# --- (1) Generator (G) ---
class Generator(nn.Module):
    def __init__(self, img_size=64, style_dim=64, max_conv_dim=512):
        super().__init__()
        dim_in = 64  # 경량화를 위한 시작 필터 수
        self.img_size = img_size

        # 1. 입력부: 흑백(1채널) 이미지를 받아서 32채널 특징 맵으로 변환
        self.from_rgb = nn.Conv2d(1, dim_in, 3, 1, 1) # Grayscale 1채널 입력

        # 2. 인코더 (Down-sampling): 형태 정보 압축
        # Down-sampling blocks (64 -> 32 -> 16 -> 8)
        self.encode = nn.ModuleList()
        curr_dim = dim_in
        for _ in range(3): # 3번 다운샘플링하여 8x8 병목 생성
            self.encode.append(ResBlock(curr_dim, curr_dim * 2))
            self.encode.append(nn.AvgPool2d(2))
            curr_dim = curr_dim * 2

        # 3. 병목 (Bottleneck): 스타일 주입 시작
        # Bottleneck (8x8 유지, AdaIN 적용)
        self.decode = nn.ModuleList()
        curr_dim = min(curr_dim, max_conv_dim)
        for _ in range(2):
            self.decode.append(AdaINResBlock(curr_dim, curr_dim, style_dim))

        # 4. 디코더 (Up-sampling): 이미지 복원 + 스타일 입히기
        # Up-sampling blocks (8 -> 16 -> 32 -> 64)
        for _ in range(3):
            self.decode.append(nn.Upsample(scale_factor=2, mode='nearest'))
            self.decode.append(AdaINResBlock(curr_dim, curr_dim // 2, style_dim))
            curr_dim = curr_dim // 2

        # 5. 출력부: 최종적으로 1채널(흑백) 이미지로 변환
        # Final Conv
        self.to_rgb = nn.Sequential(
            nn.InstanceNorm2d(curr_dim, affine=True),
            nn.LeakyReLU(0.2),
            nn.Conv2d(curr_dim, 1, 1, 1, 0) # Grayscale 1채널 출력
        )

    def forward(self, x, s):
        x = self.from_rgb(x)
        for block in self.encode:
            x = block(x)

        for block in self.decode:
            if isinstance(block, AdaINResBlock):
                x = block(x, s)
            else:
                x = block(x)

        return self.to_rgb(x)

# --- (2) Mapping Network (F) ---
class MappingNetwork(nn.Module):
    def __init__(self, latent_dim=16, style_dim=64, num_domains=10, hidden_dim=256):
        super().__init__()
        layers = []

        # 공유 레이어 (Shared)
        for _ in range(3):
            layers += [nn.Linear(latent_dim if not layers else hidden_dim, hidden_dim)]
            layers += [nn.ReLU()]
        # 1. 공유 레이어 (Shared): 모든 도메인이 공통으로 사용하는 특징 추출
        self.shared = nn.Sequential(*layers)

        # 2. 비공유 레이어 (Unshared): 각 도메인(T-shirt, Pants...)별 전용 스타일 생성기
        # 도메인별 출력 레이어 (Unshared)
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared.append(nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, style_dim)
            ))

    def forward(self, z, y):
        h = self.shared(z)
        out = []
        for layer in self.unshared:
            out += [layer(h)]
        out = torch.stack(out, dim=1) # (batch, num_domains, style_dim)

        # 사용자가 요청한 도메인(y)에 해당하는 스타일만 쏙 뽑아서 리턴
        idx = torch.arange(y.size(0)).to(y.device)
        s = out[idx, y] # 해당 도메인의 스타일 코드만 선택
        return s

# --- (3) Style Encoder (E) ---
class StyleEncoder(nn.Module):
    def __init__(self, img_size=64, style_dim=64, num_domains=10):
        super().__init__()
        dim_in = 32
        blocks = []
        blocks += [nn.Conv2d(1, dim_in, 3, 1, 1)] # 1ch input

        curr_dim = dim_in
        # Downsample to small size (64 -> 8)
        for _ in range(3): # 64 -> 32 -> 16 -> 8
            blocks += [nn.LeakyReLU(0.2)]
            blocks += [nn.Conv2d(curr_dim, curr_dim * 2, 3, 2, 1)]
            curr_dim = curr_dim * 2

        blocks += [nn.LeakyReLU(0.2)]
        blocks += [nn.Conv2d(curr_dim, curr_dim, 8, 1, 0)] # 8x8 -> 1x1
        # 1. 공유 레이어: 이미지를 보며 특징을 추출 (CNN 구조)
        # 64 -> 32 -> 16 -> 8 로 줄어들며 추상적인 특징을 잡아냄
        self.shared = nn.Sequential(*blocks)

        # 2. 비공유 레이어: 추출된 특징을 보고 "이건 바지 스타일로는 s_pants, 티셔츠로는 s_shirt야" 라고 해석
        # 도메인별 Style Code 출력
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared.append(nn.Linear(curr_dim, style_dim))

    def forward(self, x, y):
        # 이미지 x에서 시각적 특징 추출
        h = self.shared(x) # (batch, curr_dim, 1, 1)
        h = h.view(h.size(0), -1)

        out = []
        for layer in self.unshared:
            out += [layer(h)]
        out = torch.stack(out, dim=1)

        idx = torch.arange(y.size(0)).to(y.device)
        s = out[idx, y]
        return s

# --- (4) Discriminator (D) ---
class Discriminator(nn.Module):
    def __init__(self, img_size=64, num_domains=10):
        super().__init__()
        dim_in = 32
        blocks = []
        blocks += [nn.Conv2d(1, dim_in, 3, 1, 1)] # 1ch input

        curr_dim = dim_in
        # Downsample to small size (64 -> 8)
        for _ in range(3): # 64 -> 32 -> 16 -> 8
            blocks += [nn.LeakyReLU(0.2)]
            blocks += [nn.Conv2d(curr_dim, curr_dim * 2, 3, 2, 1)]
            curr_dim = curr_dim * 2

        blocks += [nn.LeakyReLU(0.2)]
        blocks += [nn.Conv2d(curr_dim, curr_dim, 8, 1, 0)] # 8x8 -> 1x1
        # 1. 공유 레이어: 이미지가 진짜인지 가짜인지 판단하기 위한 단서(특징) 추출
        # ResBlock이나 Conv 레이어를 사용하여 이미지를 분석함
        self.shared = nn.Sequential(*blocks)

        # 2. 멀티 태스크 헤드: 각 도메인별로 진위 여부를 따로 판별
        # 도메인별 진위 판별 헤드
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared.append(nn.Linear(curr_dim, 1))

    def forward(self, x, y):
        h = self.shared(x)
        h = h.view(h.size(0), -1)

        out = []
        for layer in self.unshared:
            out += [layer(h)]
        out = torch.stack(out, dim=1) # (batch, num_domains, 1)

        idx = torch.arange(y.size(0)).to(y.device)
        score = out[idx, y]
        return score

Overwriting model.py


3. solver.py (메인 실행 및 학습 루프)

In [9]:
import os
import time
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.utils import save_image, make_grid
from torch.autograd import grad as torch_grad # R1 정규화에 필요

# 분리된 파일에서 모듈 가져오기
from config import get_config, prepare_environment, get_data_transform, get_domain_labels
from model import Generator, MappingNetwork, StyleEncoder, Discriminator


class Solver:
    def __init__(self, args, device):
        self.args = args
        self.device = device
        self.save_dir = prepare_environment(args)
        self.domain_labels = get_domain_labels()

        # 데이터셋 준비 (FashionMNIST)
        transform = get_data_transform(args.img_size)

        # 학습용 데이터 로더
        dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
        self.loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True, num_workers=8, drop_last=True)

        # 평가용 데이터 로더 (FID/LPIPS 계산 시 사용될 예정)
        self.eval_loader = self.get_eval_loader(transform)

        # 모델 초기화
        # Generator의 채널 용량이 model.py에서 64/512로 증가했다고 가정하고 초기화
        self.nets = {
            'G': Generator(args.img_size, args.style_dim),
            'F': MappingNetwork(args.latent_dim, args.style_dim, args.num_domains, args.hidden_dim),
            'E': StyleEncoder(args.img_size, args.style_dim, args.num_domains),
            'D': Discriminator(args.img_size, args.num_domains)
        }

        for name, module in self.nets.items():
            module.to(self.device)
            module.train()

        # 옵티마이저
        self.optims = {
            'G': torch.optim.Adam(self.nets['G'].parameters(), lr=args.lr, betas=(0.0, 0.99)),
            'F': torch.optim.Adam(self.nets['F'].parameters(), lr=args.lr*0.01, betas=(0.0, 0.99)),
            'E': torch.optim.Adam(self.nets['E'].parameters(), lr=args.lr, betas=(0.0, 0.99)),
            'D': torch.optim.Adam(self.nets['D'].parameters(), lr=args.lr, betas=(0.0, 0.99))
        }

        # 체크포인트 로드
        self.start_iter = 0
        if args.resume_iter > 0:
            self.load_checkpoint(args.resume_iter)
            self.start_iter = args.resume_iter

    def get_eval_loader(self, transform):
        """FID/LPIPS 계산을 위한 평가용 데이터 로더를 준비합니다."""
        dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
        return DataLoader(dataset, batch_size=self.args.batch_size, shuffle=False, num_workers=8, drop_last=False, pin_memory=True)

    def save_checkpoint(self, step):
        path = os.path.join(self.save_dir, 'checkpoints', f'{step:06d}.ckpt')
        state = {
            'nets': {name: net.state_dict() for name, net in self.nets.items()},
            'optims': {name: opt.state_dict() for name, opt in self.optims.items()},
            'step': step
        }
        torch.save(state, path)
        print(f"Saved checkpoint to {path}")

    def load_checkpoint(self, step):
        path = os.path.join(self.save_dir, 'checkpoints', f'{step:06d}.ckpt')
        if not os.path.exists(path):
            print("Checkpoint not found!")
            return

        ckpt = torch.load(path, map_location=self.device)
        for name, net in self.nets.items():
            net.load_state_dict(ckpt['nets'][name])
        for name, opt in self.optims.items():
            opt.load_state_dict(ckpt['optims'][name])
        print(f"Loaded checkpoint from {path}")

    def calculate_metrics(self, step):
        """
        FID 및 LPIPS와 같은 정량적 평가지표를 계산합니다. (Placeholder)
        """
        print(f"\n--- Iteration {step}: Evaluating Metrics ---")
        fid_score = 99.99
        print(f"FID Score: {fid_score:.4f} (낮을수록 좋음)")
        lpips_score = 0.00
        print(f"LPIPS Diversity Score: {lpips_score:.4f} (높을수록 좋음)")
        print("------------------------------------------\n")

    def r1_loss(self, d_out, x_in): # R1 Loss 함수 추가
        """Discriminator의 R1 Gradient Penalty를 계산합니다."""
        grad_dout = torch_grad(
            outputs=d_out.sum(), inputs=x_in,
            create_graph=True, retain_graph=True, only_inputs=True
        )[0]
        grad_dout2 = grad_dout.pow(2)
        assert(grad_dout2.size() == x_in.size())
        r1_loss = grad_dout2.reshape(x_in.size(0), -1).sum(1).mean(0)
        return r1_loss

    def train(self):
        print("--- 학습 시작 ---")
        nets = self.nets
        optims = self.optims
        args = self.args

        data_iter = iter(self.loader)

        start_time = time.time()
        for i in range(self.start_iter, args.total_iters):

            # D를 G보다 d_train_repeats 만큼 더 학습시킵니다.
            for d_repeat in range(args.d_train_repeats): # <-- D 반복 학습 루프 시작 (이 부분이 이전 코드에 없었습니다!)

                # 1. 데이터 가져오기
                try:
                    x_real, y_org = next(data_iter)
                except StopIteration:
                    data_iter = iter(self.loader)
                    x_real, y_org = next(data_iter)

                x_real = x_real.to(self.device)
                y_org = y_org.to(self.device)

                # R1 Loss 계산을 위해 x_real에 그래디언트 추적 활성화
                x_real.requires_grad_(True)

                # 타겟 도메인 및 Latent 생성
                y_trg = torch.randint(0, args.num_domains, (x_real.size(0),)).to(self.device)
                z_trg = torch.randn(x_real.size(0), args.latent_dim).to(self.device)

                # =================================================================================== #
                #                               1. Discriminator 학습                                 #
                # =================================================================================== #

                # Real Loss
                d_out_real = nets['D'](x_real, y_org)
                d_loss_real = torch.mean(F.relu(1.0 - d_out_real))

                # R1 Regularization Loss 계산
                d_loss_r1 = self.r1_loss(d_out_real, x_real)

                # Fake Loss (Latent 기반 생성)
                with torch.no_grad():
                    s_trg = nets['F'](z_trg, y_trg)
                    x_fake = nets['G'](x_real, s_trg)

                d_out_fake = nets['D'](x_fake.detach(), y_trg)
                d_loss_fake = torch.mean(F.relu(1.0 + d_out_fake))

                # D Total Loss: Hinge Loss + R1 Loss
                d_loss = d_loss_real + d_loss_fake + args.lambda_r1 * d_loss_r1

                optims['D'].zero_grad()
                d_loss.backward()
                optims['D'].step()

                # 그래디언트 추적 해제
                x_real.requires_grad_(False)

            # G 학습은 1번만 수행
            # =================================================================================== #
            #                     2. Generator, Mapping, Encoder 학습                             #
            # =================================================================================== #

            # G 학습에 사용할 z_trg, z_trg2는 여기서 생성
            z_trg = torch.randn(x_real.size(0), args.latent_dim).to(self.device)
            z_trg2 = torch.randn(x_real.size(0), args.latent_dim).to(self.device)

            # Adversarial Loss
            s_trg = nets['F'](z_trg, y_trg)
            x_fake = nets['G'](x_real, s_trg)
            d_out_fake = nets['D'](x_fake, y_trg)
            g_loss_adv = -torch.mean(d_out_fake)

            # Style Reconstruction Loss
            s_pred = nets['E'](x_fake, y_trg)
            g_loss_sty = torch.mean(torch.abs(s_trg - s_pred))

            # Diversity Sensitive Loss
            s_trg2 = nets['F'](z_trg2, y_trg)
            x_fake2 = nets['G'](x_real, s_trg2)
            g_loss_ds = torch.mean(torch.abs(x_fake - x_fake2))

            # Cycle Consistency Loss
            s_org = nets['E'](x_real, y_org)
            x_rec = nets['G'](x_fake, s_org)
            g_loss_cyc = torch.mean(torch.abs(x_real - x_rec))

            # Total Loss
            g_loss = g_loss_adv \
                     + args.lambda_sty * g_loss_sty \
                     - args.lambda_ds * g_loss_ds \
                     + args.lambda_cyc * g_loss_cyc

            optims['G'].zero_grad()
            optims['F'].zero_grad()
            optims['E'].zero_grad()
            g_loss.backward()
            optims['G'].step()
            optims['F'].step()
            optims['E'].step()

            # =================================================================================== #
            #                                 3. 로깅 및 저장                                     #
            # =================================================================================== #

            if (i + 1) % 100 == 0:
                elapsed = time.time() - start_time
                print(f"Iter [{i+1}/{args.total_iters}] Time: {elapsed:.2f}s | "
                      f"D_loss: {d_loss.item():.4f} | G_adv: {g_loss_adv.item():.4f} | "
                      f"Sty: {g_loss_sty.item():.4f} | Cyc: {g_loss_cyc.item():.4f}")

            if (i + 1) % args.sample_freq == 0:
                self.save_samples(x_real, y_org, i + 1)

            if (i + 1) % args.save_freq == 0:
                self.save_checkpoint(i + 1)
                # self.calculate_metrics(i + 1) # 메트릭 계산 (필요 시 주석 해제)

    def save_samples(self, x_real, y_org, step):
        """학습 중간 결과 이미지 저장 (도메인 라벨 포함 시각화 개선)"""
        nets = self.nets
        args = self.args

        with torch.no_grad():
            nets['G'].eval()
            nets['F'].eval()

            x_real_subset = x_real[:args.num_domains].to(self.device)
            # y_org_subset = y_org[:args.num_domains].cpu().numpy()

            z_fix = torch.randn(1, args.latent_dim).repeat(args.num_domains, 1).to(self.device)
            y_fix = torch.arange(args.num_domains).to(self.device)
            s_fix = nets['F'](z_fix, y_fix)

            images = []

            # 1. 첫 번째 행: 소스 이미지
            source_row = [x_real_subset[i].cpu() for i in range(len(x_real_subset))]
            images.extend(source_row)

            # 2. 나머지 영역: 변환된 이미지 (스타일 변환 매트릭스)
            for j in range(args.num_domains):
                s_curr = s_fix[j].unsqueeze(0).repeat(x_real_subset.size(0), 1)
                x_fake_row = nets['G'](x_real_subset, s_curr)
                images.extend([x_fake_row[i].cpu() for i in range(len(x_real_subset))])

            images = torch.stack(images, dim=0)

            path = os.path.join(self.save_dir, 'samples', f'{step:06d}_grid.jpg')
            save_image(images, path, nrow=len(x_real_subset), padding=2, normalize=True)
            print(f"Sample image grid saved to {path}")

        # 다시 학습 모드
        nets['G'].train()
        nets['F'].train()

if __name__ == '__main__':
    # 시드 고정
    torch.manual_seed(777)
    np.random.seed(777)

    # 설정 로드
    config = get_config()

    # 장치 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")

    # Solver 시작
    solver = Solver(config, device)
    solver.train()

Device: cuda
--- 환경 설정 중 ---
저장 경로: /content/drive/MyDrive/Colab Notebooks/GAN_assignment/GAN_FMNIST_v3-2b64
--- 학습 시작 ---
Iter [100/100000] Time: 201.52s | D_loss: 0.1940 | G_adv: 1.9942 | Sty: 0.0088 | Cyc: 0.4878


KeyboardInterrupt: 

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

# 기존 설정과 모델 불러오기
from config import get_config, get_data_transform, get_domain_labels
from model import Generator, MappingNetwork, StyleEncoder

def load_model(args, device, checkpoint_step=None):
    """저장된 체크포인트를 불러옵니다."""
    model_path = os.path.join(args.save_root, args.project_name, 'checkpoints')

    # 체크포인트 지정이 없으면 가장 마지막(최신) 파일 로드
    if checkpoint_step is None:
        ckpts = sorted([f for f in os.listdir(model_path) if f.endswith('.ckpt')])
        if not ckpts:
            raise FileNotFoundError("체크포인트가 없습니다!")
        latest_ckpt = ckpts[-1]
    else:
        latest_ckpt = f'{checkpoint_step:06d}.ckpt'

    ckpt_path = os.path.join(model_path, latest_ckpt)
    print(f"Loading checkpoint: {ckpt_path}")

    ckpt = torch.load(ckpt_path, map_location=device)

    # 모델 초기화 및 가중치 로드
    generator = Generator(args.img_size, args.style_dim).to(device)
    mapping_network = MappingNetwork(args.latent_dim, args.style_dim, args.num_domains, args.hidden_dim).to(device)
    # Style Encoder는 Reference Guided Synthesis 할 때 필요 (여기서는 Latent Guided만 시연)

    generator.load_state_dict(ckpt['nets']['G'])
    mapping_network.load_state_dict(ckpt['nets']['F'])

    generator.eval()
    mapping_network.eval()

    return generator, mapping_network

def inference(args, device):
    # 1. 모델 로드
    generator, mapping_net = load_model(args, device)

    # 2. 테스트 데이터 로드 (학습에 안 쓴 데이터)
    transform = get_data_transform(args.img_size)
    test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True) # 10장만 샘플링

    # 3. 소스 이미지 가져오기
    x_real, y_org = next(iter(test_loader))
    x_real = x_real.to(device)

    # 4. 시각화 준비
    domain_labels = get_domain_labels()

    plt.figure(figsize=(15, 10))

    # [Row 1] 원본 이미지 (Source)
    for i in range(10):
        plt.subplot(11, 10, i + 1)
        img = x_real[i].cpu().squeeze().numpy()
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        if i == 0: plt.title("Source", fontsize=12, loc='left')

    # [Rows 2-11] 각 도메인으로 스타일 변환 (Latent Guided)
    # 고정된 Random Noise z 하나를 모든 도메인에 적용해 봅니다.
    z_trg = torch.randn(1, args.latent_dim).to(device)

    for row_idx in range(args.num_domains): # 0~9 (각 도메인별)
        # 해당 도메인(row_idx)의 스타일 코드 생성
        y_trg = torch.tensor([row_idx]).to(device)
        s_trg = mapping_net(z_trg, y_trg) # (1, style_dim)

        # 스타일 코드를 배치 크기만큼 복사 (1 -> 10)
        s_trg = s_trg.repeat(10, 1)

        # 이미지 생성
        with torch.no_grad():
            x_fake = generator(x_real, s_trg)

        # 결과 출력
        for col_idx in range(10):
            plt.subplot(11, 10, (row_idx + 1) * 10 + col_idx + 1)
            img = x_fake[col_idx].cpu().squeeze().numpy()
            plt.imshow(img, cmap='gray')
            plt.axis('off')

            # 왼쪽 첫 열에만 도메인 이름 표시
            if col_idx == 0:
                plt.text(-10, 32, domain_labels[row_idx], fontsize=10, va='center')

    plt.tight_layout()
    plt.show()

# 실행
if __name__ == '__main__':
    config = get_config()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    inference(config, device)